Weitere Schritte
- Doc-String dort wo es einen braucht einbauen, Code muss strukturiert sein. 
- Client soll beendet werden können. In sendMessage noch ein if einbauen —> vielleicht mit einem Exit keyboard?
- Auswählen an welche Person man noch schreiben möchte
- URI Attribut anschauen? http Pfad? (eher unwichtig) (nice to have, keine prio)

In [5]:
%gui asyncio

from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display
import asyncio
import os
import websockets
from re import search

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
    widget.on_click(getvalue)
    return future


class Client:
    
    def __init__(self, user_name, server_port=8080):
        self.user_name = user_name
        self.server_port = server_port
        self.chat_input = widgets.Text(placeholder='Send your message like this --> sender:receiver:message',description='Chat Input:',layout=Layout(width='80%'))
        self.chat_area = widgets.Textarea(value = '', disabled=True, description='Chat:', layout=Layout(width='99%', height='200px'))
        self.chat_button = widgets.Button(description='Send!',layout=Layout(width='19%'))
        print(f"this is my user name:  {self.user_name}\n")
    
    async def run(self):
        self.websocket = await websockets.connect(f"ws://{self.user_name}:{self.server_port}")
        # The first message is the user name
        self.chat_area.value += "Connection was successful for client at " + f"ws://{self.user_name}:{self.server_port}\n"
        await self.websocket.send(os.path.expandvars("$USER"))
        asyncio.create_task(self.send_message())
        asyncio.create_task(self.receive_message())
        display(VBox([HBox([self.chat_input, self.chat_button]), self.chat_area]))

    async def send_message(self):
        while True:
            x = await wait_for_change(self.chat_button)
            self.chat_area.value += f"{self.chat_input.value}\n"
            if search('exit', self.chat_input.value):
                await self.websocket.close()
                self.chat_area.value += "Connection was successfully closed."
            else:
                await self.websocket.send(self.chat_input.value)
                self.chat_input.value = ""

    async def receive_message(self):
        while True:
            response = await self.websocket.recv()
            self.chat_area.value += f"{response}\n"
    

In [6]:
client = Client(user_name='julia.mueller')
#client = Client(user_name='michelle.ngu')
await client.run()

this is my user name:  julia.mueller



In [ ]:
michelle.ngu:julia.mueller:hoi
michelle.ngu:*:hoi
